<a href="https://colab.research.google.com/github/thisissamuca/GOES_16/blob/main/Getting_Mask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Montando drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Máscara de nuvem

In [ ]:
from affine import Affine
import os
import netCDF4
import cv2
import numpy as np
import xarray as xr
import rasterio
from satpy import Scene
from typing import List, Dict, Optional, Generator
import warnings
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

# Configuração para suprimir avisos não essenciais
warnings.filterwarnings('ignore')

class NetCDFBatchProcessor:
    """Classe para processamento em lote de arquivos NetCDF com dados de satélite."""

    def __init__(self, base_path: str = '/content/drive/MyDrive/NBR'):
        self.base_path = base_path

    def list_product_files(self, product: str) -> List[str]:
        """Lista todos os arquivos disponíveis para um produto específico."""
        product_path = os.path.join(self.base_path, product, 'netCDF')
        return sorted([f for f in os.listdir(product_path) if f.endswith('.nc')])

    def get_variables(self, product: str, filename: str) -> Dict[int, str]:
        """Retorna um dicionário com as variáveis disponíveis em um arquivo NetCDF."""
        file_path = os.path.join(self.base_path, product, 'netCDF', filename)

        with netCDF4.Dataset(file_path) as dataset:
            variables = {i: var for i, var in enumerate(dataset.variables.keys())}

        return variables

    def load_dataset(self, product: str, filename: str) -> xr.Dataset:
        """Carrega um arquivo NetCDF como xarray Dataset."""
        file_path = os.path.join(self.base_path, product, 'netCDF', filename)
        return xr.open_dataset(file_path)

class ImageBatchProcessor:
    """Classe para processamento em lote de imagens derivadas de dados de satélite."""

    @staticmethod
    def reclassify_array(data: np.ndarray, mapping: Dict[int, int]) -> np.ndarray:
        """Reclassifica um array numpy baseado em um dicionário de mapeamento (versão otimizada)."""
        # Versão otimizada usando np.where para melhor desempenho
        result = np.zeros_like(data, dtype=np.float32)
        for key, value in mapping.items():
            result = np.where(data == key, value, result)
        return result

    @staticmethod
    def safe_convert_to_float(data: np.ndarray) -> np.ndarray:
        """Converte valores do array para float de forma segura (versão vetorizada)."""
        # Versão otimizada usando vectorize com cache
        float_func = np.vectorize(lambda x: float(x) if isinstance(x, (int, float)) else np.nan,
                                otypes=[np.float32])
        return float_func(data)

    @staticmethod
    def resize_image(image: np.ndarray, target_shape: tuple = (5424, 5424)) -> np.ndarray:
        """Redimensiona imagem mantendo proporções (usando interpolação bilinear)."""
        return cv2.resize(image, target_shape, interpolation=cv2.INTER_LINEAR)

class GeoTIFFBatchExporter:
    """Classe para exportação em lote de dados para formato GeoTIFF."""

    @staticmethod
    def create_transform(area) -> Affine:
        """Cria transformação afim a partir da área de dados."""
        res_x = area.pixel_size_x
        res_y = -area.pixel_size_y
        xmin, ymax = area.area_extent[0], area.area_extent[3]
        return Affine.translation(xmin, ymax) * Affine.scale(res_x, res_y)

    @staticmethod
    def export_to_geotiff(data: np.ndarray, crs, transform, output_path: str):
        """Exporta array numpy para arquivo GeoTIFF."""
        profile = {
            'driver': 'GTiff',
            'height': data.shape[0],
            'width': data.shape[1],
            'count': 1,
            'dtype': data.dtype,
            'crs': crs,
            'transform': transform,
            'compress': 'lzw',  # Adicionado compressão para reduzir tamanho dos arquivos
            'tiled': True      # Melhora performance para arquivos grandes
        }

        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        with rasterio.open(output_path, 'w', **profile) as dst:
            dst.write(data, 1)

def process_single_file(ncdf_processor, img_processor, exporter, product: str, filename: str,
                       reclass_map: Dict, target_shape: tuple, output_dir: str):
    """Processa um único arquivo NetCDF e exporta para GeoTIFF."""
    try:
        start_time = time.time()

        # Carrega e processa os dados
        ds = ncdf_processor.load_dataset(product, filename)
        first_var = list(ds.variables.keys())[0]
        variable_data = ds[first_var].data

        # Processamento da imagem
        reclassified = img_processor.reclassify_array(variable_data, reclass_map)
        float_array = img_processor.safe_convert_to_float(reclassified)
        resized_image = img_processor.resize_image(float_array, target_shape)

        # Configuração do sistema de coordenadas
        file_path = os.path.join(ncdf_processor.base_path, product, 'netCDF', filename)
        scn = Scene(filenames=[file_path], reader='abi_l2_nc')
        scn.load(['BCM'])

        area = scn['BCM'].attrs['area']
        transform = exporter.create_transform(area)
        crs = area.to_cartopy_crs()

        # Gera nome do arquivo de saída
        output_filename = f"{os.path.splitext(filename)[0]}_processed.tif"
        output_path = os.path.join(output_dir, output_filename)

        # Exportação
        exporter.export_to_geotiff(resized_image, crs, transform, output_path)

        processing_time = time.time() - start_time
        return (filename, True, processing_time)

    except Exception as e:
        return (filename, False, str(e))

def batch_process_files(product: str, output_dir: str, max_workers: int = 4):
    """Processa todos os arquivos de um produto em paralelo."""
    # Configurações
    RECLASS_MAP = {1: 0, 0: 1}
    TARGET_SHAPE = (5424, 5424)

    # Inicialização dos processadores
    ncdf_processor = NetCDFBatchProcessor()
    img_processor = ImageBatchProcessor()
    exporter = GeoTIFFBatchExporter()

    # Obtém lista de arquivos
    files = ncdf_processor.list_product_files(product)
    if not files:
        raise FileNotFoundError(f"Nenhum arquivo encontrado para o produto {product}")

    print(f"Iniciando processamento de {len(files)} arquivos...")

    # Cria diretório de saída se não existir
    os.makedirs(output_dir, exist_ok=True)

    # Processamento em paralelo
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = []
        for filename in files:
            future = executor.submit(
                process_single_file,
                ncdf_processor,
                img_processor,
                exporter,
                product,
                filename,
                RECLASS_MAP,
                TARGET_SHAPE,
                output_dir
            )
            futures.append(future)

        # Monitora progresso e resultados
        success_count = 0
        for future in as_completed(futures):
            filename, status, result = future.result()
            if status is True:
                print(f"Arquivo {filename} processado com sucesso em {result:.2f} segundos")
                success_count += 1
            else:
                print(f"Erro ao processar {filename}: {result}")

    print(f"\nProcessamento concluído! {success_count}/{len(files)} arquivos processados com sucesso.")

if __name__ == "__main__":
    # Configurações para execução
    PRODUCT = "ABI-L2-ACMF"
    OUTPUT_DIR = "/content/drive/MyDrive/NBR/processed_output"
    MAX_WORKERS = 1  # Ajuste conforme os recursos disponíveis

    # Executa o processamento em lote
    start_total = time.time()
    batch_process_files(PRODUCT, OUTPUT_DIR, MAX_WORKERS)
    total_time = time.time() - start_total

    print(f"\nTempo total de processamento: {total_time:.2f} segundos")

Iniciando processamento de 300 arquivos...
Arquivo OR_ABI-L2-ACMF-M6_G16_s20201521600164_e20201521609472_c20201521610396.nc processado com sucesso em 12.51 segundos
Arquivo OR_ABI-L2-ACMF-M6_G16_s20201531600167_e20201531609475_c20201531610372.nc processado com sucesso em 11.55 segundos
Arquivo OR_ABI-L2-ACMF-M6_G16_s20201541600171_e20201541609479_c20201541610314.nc processado com sucesso em 11.40 segundos
Arquivo OR_ABI-L2-ACMF-M6_G16_s20201551600172_e20201551609480_c20201551610365.nc processado com sucesso em 48.81 segundos
Arquivo OR_ABI-L2-ACMF-M6_G16_s20201561600173_e20201561609481_c20201561610298.nc processado com sucesso em 11.43 segundos
Arquivo OR_ABI-L2-ACMF-M6_G16_s20201571600175_e20201571609483_c20201571610369.nc processado com sucesso em 12.23 segundos
Arquivo OR_ABI-L2-ACMF-M6_G16_s20201581600175_e20201581609483_c20201581610216.nc processado com sucesso em 11.38 segundos
Arquivo OR_ABI-L2-ACMF-M6_G16_s20201591600177_e20201591609485_c20201591610388.nc processado com sucesso

# Máscara hídrica

In [ ]:
import numpy as np
import rasterio
import xarray as xr
import cv2
from pathlib import Path
from typing import List, Tuple, Optional, Dict
import concurrent.futures
import os
import re

def binarize_non_nan(array: np.ndarray) -> np.ndarray:
    """Converte valores não-NaN de um array para 1, mantendo NaNs."""
    return np.where(np.isnan(array), np.nan, 1)

def load_and_preprocess_tif(tif_path: str) -> Tuple[np.ndarray, dict]:
    """Carrega e pré-processa imagem TIFF."""
    with rasterio.open(tif_path) as src:
        image = src.read(1)
        profile = src.profile.copy()
        image[image == 0] = np.nan
    return image, profile

def load_and_resize_nc(nc_path: str, target_shape: Tuple[int, int]) -> np.ndarray:
    """Carrega e redimensiona dados de arquivo NetCDF."""
    with xr.open_dataset(nc_path) as ds:
        first_var = next(iter(ds.variables))
        return cv2.resize(ds[first_var].data, (target_shape[1], target_shape[0]),
                        interpolation=cv2.INTER_LINEAR)

def save_tif(data: np.ndarray, output_path: str, profile: dict) -> None:
    """Salva dados como arquivo TIFF."""
    with rasterio.open(output_path, 'w', **profile) as dst:
        dst.write(data, 1)

def process_pair(tif_path: str, nc_path: str, output_dir: str, prefix: str = "") -> Tuple[str, str]:
    """Processa um par de arquivos TIFF e NetCDF."""
    Path(output_dir).mkdir(parents=True, exist_ok=True)

    base_name = Path(tif_path).stem
    output_path1 = os.path.join(output_dir, f"{prefix}_{base_name}.tif")
    tif_image, profile = load_and_preprocess_tif(tif_path)
    target_shape = tif_image.shape

    # Processamento do primeiro arquivo
    nc_data = load_and_resize_nc(nc_path, target_shape)
    result = tif_image * nc_data
    result[result < 0.15] = np.nan

    save_tif(binarize_non_nan(result), output_path1, profile)

def filter_nc_files(nc_files: List[str], pattern: str = "M6C03") -> List[str]:
    """Filtra arquivos NC contendo o padrão especificado no nome."""
    return [f for f in nc_files if re.search(pattern, str(f), re.IGNORECASE)]

def match_tif_nc_pairs(tif_files: List[str], nc_files: List[str]) -> List[Tuple[str, str]]:
    """Emparelha arquivos TIFF e NC por data ou outro critério."""
    # Implementação básica - assume mesma ordem e quantidade
    # Para implementação mais robusta, adicione lógica de correspondência por data/timestamp
    return list(zip(tif_files, nc_files))

def batch_process(tif_files: List[str], nc_files: List[str], output_dir: str,
                 max_workers: Optional[int] = None, prefix: str = "",
                 nc_pattern: str = "M6C03") -> Dict[str, Tuple[str, str]]:
    """Processa lotes de arquivos TIFF e NetCDF (com filtro por padrão no NC)."""
    # Filtra arquivos NC
    filtered_nc = filter_nc_files(nc_files, nc_pattern)

    # Emparelha arquivos
    pairs = match_tif_nc_pairs(tif_files, filtered_nc)

    if not pairs:
        raise ValueError("Nenhum par de arquivos válido encontrado após filtragem")

    results = {}

    with concurrent.futures.ProcessPoolExecutor(max_workers=max_workers) as executor:
        futures = {
            executor.submit(process_pair, tif, nc, output_dir, prefix): (tif, nc)
            for tif, nc in pairs
        }

        for future in concurrent.futures.as_completed(futures):
            tif, nc = futures[future]
            try:
                output1, output2 = future.result()
                results[f"{Path(tif).stem}_{Path(nc).stem}"] = (output1, output2)
            except Exception as e:
                print(f"Erro ao processar {tif} e {nc}: {str(e)}")

    return results

# Exemplo de uso
if __name__ == "__main__":
    input_dir_nc = "/content/drive/MyDrive/NBR/ABI-L2-CMIPF/netCDF"
    input_dir_tif = "/content/drive/MyDrive/NBR/processed_output"
    output_dir = "/content/drive/MyDrive/NBR/MASK_CLOUD_WATER_G16"
    prefix = "MASK_"

    # Lista e filtra arquivos
    tif_files = sorted(Path(input_dir_tif).glob("*.tif"))
    nc_files = sorted(Path(input_dir_nc).glob("*.nc"))

    tif_paths = [str(f) for f in tif_files]
    nc_paths = [str(f) for f in nc_files]

    # Processa em lote (com filtro para M6C03)
    resultados = batch_process(
        tif_paths,
        nc_paths,
        output_dir,
        max_workers=1,
        prefix=prefix,
        nc_pattern="M6C03"
    )

    print(f"Processamento concluído. {len(resultados)} pares processados.")
    for pair, outputs in resultados.items():
        print(f"{pair} -> {outputs[0]}, {outputs[1]}")

Erro ao processar /content/drive/MyDrive/NBR/processed_output/OR_ABI-L2-ACMF-M6_G16_s20201521600164_e20201521609472_c20201521610396_processed.tif e /content/drive/MyDrive/NBR/ABI-L2-CMIPF/netCDF/OR_ABI-L2-CMIPF-M6C03_G16_s20201521600164_e20201521609472_c20201521609565.nc: cannot unpack non-iterable NoneType object
Erro ao processar /content/drive/MyDrive/NBR/processed_output/OR_ABI-L2-ACMF-M6_G16_s20201531600167_e20201531609475_c20201531610372_processed.tif e /content/drive/MyDrive/NBR/ABI-L2-CMIPF/netCDF/OR_ABI-L2-CMIPF-M6C03_G16_s20201531600167_e20201531609475_c20201531609565.nc: cannot unpack non-iterable NoneType object
Erro ao processar /content/drive/MyDrive/NBR/processed_output/OR_ABI-L2-ACMF-M6_G16_s20201541600171_e20201541609479_c20201541610314_processed.tif e /content/drive/MyDrive/NBR/ABI-L2-CMIPF/netCDF/OR_ABI-L2-CMIPF-M6C03_G16_s20201541600171_e20201541609479_c20201541609570.nc: cannot unpack non-iterable NoneType object
Erro ao processar /content/drive/MyDrive/NBR/process